- https://www.nltk.org/howto/wordnet.html WordNet examples

In [4]:
from nltk.corpus import wordnet as wn
import numpy as np
from scipy import stats
import math

#### Funzione per trovare tutti gli iperonimi di un synset e salvarli in una lista di liste
 [ [iperonimi di livello1, ...], [iperonimi di livello 2, ...], ... , [...] ]

In [13]:
def get_all_hypernyms(synset):
    ret_list = []
    temp_list = []
    hypernyms = synset.hypernyms()
    while hypernyms:
        for hyper in hypernyms:
            temp_list.append(hyper)
        ret_list.append(temp_list)
        temp_list = []
        hypernyms = hypernyms[0].hypernyms()
    return ret_list


print(get_all_hypernyms(wn.synsets('dog')[0]))

[[Synset('canine.n.02'), Synset('domestic_animal.n.01')], [Synset('carnivore.n.01')], [Synset('placental.n.01')], [Synset('mammal.n.01')], [Synset('vertebrate.n.01')], [Synset('chordate.n.01')], [Synset('animal.n.01')], [Synset('organism.n.01')], [Synset('living_thing.n.01')], [Synset('whole.n.02')], [Synset('object.n.01')], [Synset('physical_entity.n.01')], [Synset('entity.n.01')]]


#### Funzione che calcola LCS
Risalendo la gerarchia degli iperonimi dei due sensi, trovo il primo antenato comune

In [136]:
def my_lowest_common_subsumer(syn1, syn2):
    syn1_hypernyms = sum(get_all_hypernyms(syn1), [])
    syn2_hypernyms = sum(get_all_hypernyms(syn2), [])

    for h1 in syn1_hypernyms:
        if h1 in syn2_hypernyms:
            return h1

In [137]:
print(wn.synsets('dog')[1].lowest_common_hypernyms(wn.synsets('happiness')[1]))
print(my_lowest_common_subsumer(wn.synsets('dog')[1], wn.synsets('happiness')[1]))

[Synset('entity.n.01')]
Synset('entity.n.01')


#### Metodo che restituisce la profondità del synset

In [138]:
def depth_synset(syn):
    if syn is not None:
        return 1 if syn.hypernyms() == [] else 1 + depth_synset(syn.hypernyms()[0])
    else:
        return 0

#### Metodo per calcolare la profondità massima
Restituisce sempre 19, per velocizzare le esecuzioni salviamo il valore in una costante (DEPTH_MAX)

In [139]:
def max_path(): 
    max_path = 0
    for synset in wn.all_synsets():
        if synset.max_depth() > max_path:
            max_path = synset.max_depth()
    return max_path

In [140]:
DEPTH_MAX = 19

### Funzione che calcola la distanza tra due synset
1. Salviamo gli iperonimi di entrambi i sensi, sia a livelli che come lista
2. Per entrambe le liste di iperonimi (s1_hyper, s2_hyper):
    
    a. scorro tutti i livelli della lista [ [iperonimi di livello1, ...], [iperonimi di livello 2, ...], ... ]
    
    b. scorro tutti gli iperonimi per ogni livello
    
    c. per ogni iperonimo, se è presente nella lista di iperonimi del **secondo synset in input**, allora quello è l'antenato comune.
        Più nel dettaglio, se il contatore vale 0, cioè non abbiamo ancora incontrato nessun'altro antenato comune, lo incremento con il valore di nodi di distanza dal **primo synset in input**. Ciò significa che il synset in questione è un antenato comune
3. Restituisco la somma dei contatori


In [14]:
def my_distance_between_syn(syn_1, syn_2):
    if syn_1 == syn_2:
        return 0
    
    s1_hyper = get_all_hypernyms(syn_1)
    s2_hyper = get_all_hypernyms(syn_2)

    s1_temp = sum(s1_hyper, []) 
    s2_temp = sum(s2_hyper, []) 
    

    c_1 = 0
    for i in range(0, len(s1_hyper)):
        for j in range(0, len(s1_hyper[i])):
            if s1_hyper[i][j] in s2_temp and c_1 == 0:
                c_1 += i+1

    c_2 = 0
    for i in range(0, len(s2_hyper)):
        for j in range(0, len(s2_hyper[i])):
            if s2_hyper[i][j] in s1_temp and c_2 == 0:
                c_2 += i+1
    
    if c_1 > 0 and c_2 > 0:
        return (c_1+c_2)
    else:
        return None


print(wn.synsets('car')[0].shortest_path_distance(wn.synsets('dog')[0]))
print(my_distance_between_syn(wn.synsets('car')[0], wn.synsets('dog')[0]))

12
17


### Wu & Palmer
Resituisce valori tra 0 e 1, dove più ci si avvicina a 1 più i sensi sono simili.

In [16]:
def wu_palmer(syn1, syn2):
    dep = 0

    lcs = my_lowest_common_subsumer(syn1, syn2)
    dep = (depth_synset(syn1) + depth_synset(syn2))
    
    if dep == 0:
        dep = 0.001
        
    return 2 * depth_synset(lcs) / dep

### Shortest Path
Valore che può oscillare tra 0 e 2 * depthMax.
Più il valore si avvicina a 2 * depthMax più i sensi sono simili, questo vuol dire che la distanza tra i due sensi (len(s1,s2)) è minima o uguale a 0

In [15]:
def shortest_path(syn1, syn2):
    return 2 * DEPTH_MAX - my_distance_between_syn(syn1, syn2) if my_distance_between_syn(syn1, syn2) is not None else 0

### Leakcock & Chodorow
I valori sono compresi tra 0 e log(2depthMax + 1)

In [17]:
def leakcock_chodorow(syn1, syn2):
    distance = my_distance_between_syn(syn1, syn2) 
    if distance is not None:
        if distance != 0:
            return -np.log(distance / 2 * DEPTH_MAX)
        else:
             return -np.log(distance+1 / (2 * DEPTH_MAX)+1)
    else:
        return 0

#### Esecuzione

Lettura delle coppie di termini dal file WordSim353.csv e recupero dei relativi synsets

In [18]:
dataset = []

with open(r'../resources/WordSim353.csv', 'r') as f:
    word_sim = f.readlines()[1:]
    for tuple in word_sim:
        dataset.append(tuple.split(','))

syns_1 = []
syns_2 = []

for tuple in dataset:
    syns_1.append(wn.synsets(tuple[0]))
    syns_2.append(wn.synsets(tuple[1]))

Calcolo della similarità, utilizzando le tre metriche implementate. Faccio il prodotto cartesiano dei synset delle due parole in input e calcolo la similarità per ogni coppia di synset. Per ogni coppia di termini, prendo il valore massimo di similarità per ogni metrica

In [146]:
wp = []
sp = []
lc = []

max_wu = 0
max_sp = 0
max_lc = 0

for i in range(len(syns_1)):
    for j in range(len(syns_1[i])):
        for k in range(len(syns_2[i])):
            
            
            wu_temp = wu_palmer(syns_1[i][j], syns_2[i][k])
            sp_temp = shortest_path(syns_1[i][j], syns_2[i][k])
            lc_temp = leakcock_chodorow(syns_1[i][j], syns_2[i][k])

            if wu_temp > float(max_wu):
                max_wu = wu_temp
            if sp_temp > float(max_sp):
                max_sp = sp_temp
            if lc_temp < float(max_lc):
                max_lc = lc_temp

    wp.append(max_wu) 
    max_wu = 0
    sp.append(max_sp)
    max_sp = 0
    lc.append(max_lc)
    max_lc = 0

### Metodi di valutazione
Lettura dei valori di similarità nel file WordSim353.csv

In [147]:
word_sim_353 = [float(data[2].strip('\n')) for data in dataset]

#### Calcolo dei valori di correlazione di Spearman e Pearson

In [148]:
print("\nSPEARMAN'S CORRELATION COEFFICIENT\n")
print(f"WU & PALMER             {stats.spearmanr(wp, word_sim_353).correlation}")
print(f"SHORTEST PATH           {stats.spearmanr(sp, word_sim_353).correlation}")
print(f"LEAKCOCK & CHODOROW     {stats.spearmanr(lc, word_sim_353).correlation}\n\n")

print("\nPEARSON'S CORRELATION COEFFICIENT\n")
print(f"WU & PALMER             {stats.pearsonr(wp, word_sim_353)}")
print(f"SHORTEST PATH           {stats.pearsonr(sp, word_sim_353)}")
print(f"LEAKCOCK & CHODOROW     {stats.pearsonr(lc, word_sim_353)}")





SPEARMAN'S CORRELATION COEFFICIENT

WU & PALMER             0.31085037768164225
SHORTEST PATH           0.27075321829782795
LEAKCOCK & CHODOROW     0.14485364030465941



PEARSON'S CORRELATION COEFFICIENT

WU & PALMER             (0.25357690588394033, 1.389583643256362e-06)
SHORTEST PATH           (0.08555125323071107, 0.10858096975665119)
LEAKCOCK & CHODOROW     (0.13391553537314543, 0.011786460824796455)
